# Basic Tutorial

This python3 notebook demonstrates the basics of using [M3's](https://github.com/mbari-media-management) video asset manager and annotation API's for your own applications. 


## Before you start

### Start the microservices needed for this notebook

To get started you will need to start both the [video asset manager](https://github.com/mbari-media-management/vampire-squid) and [annotation](https://github.com/mbari-media-management/annosaurus) microservices using [Docker](https://www.docker.com/). One of the easiest ways to do this is to use the [m3-microservices project](https://github.com/mbari-media-management/m3-microservices):

```
git clone https://github.com/mbari-media-management/m3-microservices.git
cd m3-microservices
# Edit .env as per the README
docker-compose build
docker-compose up
```

### Get your IP address

On Mac/Linux: 

```
ifconfig | grep "inet " | grep -Fv 127.0.0.1 | awk '{print $2}'
```



In [1]:
# Enter your IP address here
ipAddress = "192.168.1.85"

### Set your client secrets

Look in `m3-microservices/.env` for the values for:

- ANNO_APP_CLIENT_SECRET
- VAMP_APP_CLIENT_SECRET

and set them below. I've already set them to the default values so if you haven't changed in the `.env` file, you can skip this step

In [ ]:
annoSecret = "foo"
vamSecret = "foo"

## Base Endpoints

Using the IP address, let's build the base URLs to point to each of our services. We're using the default ports and service names that are defined in m3-microservices. You can change these to point at your production or development services and the rest of this notebook should work just fine.

In [2]:
annosaurusUrl = "http://%s:8082/anno/v1" %(ipAddress)
vampireSquidUrl = "http://%s:8084/vam/v1" % (ipAddress)

## Endpoints

Using your IP address, we need to build the URL's to the microservices so we know where to send our REST requests.

In [3]:
# Useful annosaurus endpoints
annotation_url = "%s/annotations" % (annosaurusUrl)
image_url = "%s/images" % (annosaurusUrl)
observation_url = "%s/observations" % (annosaurusUrl)
association_url = "%s/associations" % (annosaurusUrl)

# Useful vampire-squid endpoints
media_url = "%s/media" % (vampireSquidUrl)



## Helper functions

We're going to create a few helper functions to:

- Simplify making requests and parsing the responses to our microservices
- Display some informative info, so we can understand what is happening

In [4]:
import datetime
import dateutil
import json
import pprint
import random
import requests
import urllib
import uuid

def show(s, data = None):
    "Display the json response from API calls"
    pp = pprint.PrettyPrinter(indent=2)
    print("--- " + s)
    if data:
      pp.pprint(data)
    
def iso8601():
    "Standardize the date format for pretty printing"
    return datetime.datetime.now(datetime.timezone.utc).isoformat()[0:-6] + "Z"
    
def parse_response(r):
    "Parse a JSON response"
    try:
       return json.loads(r.text)
    except:
        s = "URL: %s\n%s (%s): %s" % (r.request.url, r.status_code, r.reason, r.text)
        print(s)
        return {}
    
# --- Some helper functions that display the web traffic
#     Useful for demo
def pretty_print(pr):
    print('{}\n{}\n{}\n\n{}'.format(
        '-----------START-----------',
        pr.method + ' ' + pr.url,
        '\n'.join('{}: {}'.format(k, v) for k, v in pr.headers.items()),
        pr.body,
    ))
    
def send(pr):
    pretty_print(pr)
    s = requests.Session()
    return s.send(pr)
     
def pretty_delete(url):
    r = requests.Request('DELETE', url)
    pr = r.prepare()
    return parse_response(send(pr))

def pretty_get(url):
    r = requests.Request('GET', url)
    pr = r.prepare()
    return parse_response(send(pr))

def pretty_post(url, data = {}):
    r = requests.Request('POST', url, data = data)
    pr = r.prepare()
    return parse_response(send(pr))

def pretty_put(url, data = {}):
    r = requests.Request('PUT', url, data = data)
    pr = r.prepare()
    return parse_response(send(pr))
    
    
# --- Basic REST calls, you'd probably use these in your own 
#     applications instead of the pretty-fied versions above. 
def delete(url):
    return parse_response(requests.delete(url))

def get(url):
    return parse_response(requests.get(url))
    
def post(url, data = {}):
    return parse_response(requests.post(url, data))

def put(url, data = {}):
    return parse_response(requests.put(url, data))

## Typical Usage

### Authentication

For GET reguests, where you are just retrieving information. You don't need to worry about authentication. For POST, PUT, and DELETE requests, which modify the database, you will be using authentication. (NOTE that you can disable authentication usage if you really, really like to live on the scary edge and don't value your data)

The APIs are configured to use JWT authentication. You submit a POST request using your API secret


## Typical Usage

We'll walk through a typical usage for an annotations application.

### Look up a movie of interest

In order to do this, we first need to register one in [vampire-squid](https://github.com/mbari-media-management/vampire-squid). We'll use the simplest possible call and pass the least amount of information required to register our movie. Here's the minimum info:

- __video_sequence_name__: At MBARI, this name represents all videos from a single ROV dive. We format it as `ROV_name dive_number`. For example: `Doc Ricketts 0952` or `Ventana 2345`. It can be anything that makes sense to you.
- __camera_id__: This is the identifier for the _thing_ that collected the video. We use the ROV name, or AUV name, or some other unique id to tag what collected the video.
- __video_name__: A single section of video form a dive, may have several representations. A _big_ relatively uncompressed __master__, a slightly more managebly sized __mezzanine__, and one or more highly compressed __proxies__. This name is the catch-all for all versions of the same video. I've been using `rov_name dive_number start_timestamp` as my representation as it's really easy to automate (e.g. `Ventana 3456 20171112T012345Z`), but again, use what makes sense to you.
- __uri__: This is typically the URL to your movie. Your movie really should be served off of a web server. If you don't have one, I have a docker container that can do it for you.
- __start_timestamp__: This is the moment when the first frame in the video was recorded. Usually, you can extract this from the movies metadata atoms. That is, if you remembered to correctly sync the clock on your movie recorder.
- __duration_millis__: This is actually an optional parameter, but it makes your life sooooo much easier if it's included when you register a video. This is the length (duration) of the movie in milliseconds.

In [5]:
# Register a video 
my_media = post(media_url, 
               data = {"video_sequence_name": "Ventana 0952",
                      "camera_id": "Ventana", 
                      "video_name": "Ventana 0952 - 1 of 8",
                      "uri": "http://totally.fake.org/Ventana_20171118T202801Z_master.mp4",
                      "start_timestamp": "2017-11-18T20:28:01.003Z",
                      "duration_millis": 123456})

# Look up a video. Actually, we're looking up all videos in this 
# video sequence
url = media_url + "/videosequence/" + my_media['video_sequence_name']
my_media = pretty_get(url)[0]

URL: http://192.168.1.85:8084/vam/v1/media
401 (Unauthorized): The request did not include valid authorization credentials


KeyError: 'video_sequence_name'

## Video Reference UUID

The `video_reference_uuid` is a key we need to create annotations. This tells our annotation system which video an annotation belongs. And when I say _video_ I mean the video attached to a specfic URL. Our video asset manager uses the following terminology:

- __video_sequence__: This is a grouping of related videos from a deployment of a camera. At MBARI, we chop up dives into 15 minute segments to make each file somewhat managemable in size. So for a single 8 hour dive we would have at least 32 video files grouped into a single _video_sequence_
- __video__: This is a single segment of a video_sequence. e.g. One 15 minute segment. It encapsulates 2 important things: a video_name and the start_timestamp. Note that a video may have multiple representations (i.e. master, mezzanine, and proxies). 
- __video_reference__: Each _video_ may actually represent several files. (Again ... master, mezzanine, proxies). A video_reference is a pointer to one of those files. 

## Create Your First Annotation

Again, we'll keep this very simple. Here's the minimum fields you need to provide:

- __video_reference_uuid__: You get this from your video asset manager. It's an id for the video that the annotation belongs too.
- __concept__: This is basically whatever it is you're annotating. Fish, rock, Grimpoteuthis, whatever.
- __observer__: Who made the annotation. This could be a full name, email, login id, whatever you deem appropriate for your application.
- __elapsed_time_millis__: This is the elapsed time from the start of the movie when the annotation occurred. Your video player should be able to give you this information. Note: some applications won't have elapsed_time available. For example, video tapes ... you can use __timecode__ (e.g. `01:23:45:01`) instead. Or real-time annotations, in which case you can use `recorded_timestamp` (e.g. `2017-11-09T01:23:45.01.123Z`). Note, if you have the correct `start_timestamp` (from your media you retrieved above) and `elapsed_time_millis` from your video player, you can calculate the `recorded_timestamp` and include that too. It makes your data _MUCH_ more useful!!

In [ ]:
# Index into movie when annotation occurs
elapsed_time_millis = 2000

# We can calculate the recorded_timestamp from the start_timestamp
# in my_media and elased_time_millis
import dateutil
starttime = dateutil.parser.parse(my_media['start_timestamp'])
recordedtime = starttime + datetime.timedelta(milliseconds=elapsed_time_millis)


annotation = pretty_post(annotation_url,
                 data = {"video_reference_uuid": my_movie['video_reference_uuid'],
                        "concept": "Aegina citrea", 
                        "observer", "Brian Schlining",
                        "elapsed_time_millis": "2000",
                        "recorded_timestamp": recordedtime.isoformat()})

In [ ]:
import dateutil
d = dateutil.parser.parse('2008-09-03T20:56:35.450686Z')
d.isoformat()